In [60]:
import os
import sys
path = os.path.abspath(os.path.curdir)
while not path.endswith('sbc'):
    path = os.path.dirname(path)
sys.path.insert(0, path)
import pandas as pd
from SberBusinessCampaign import *

fr = FileReader()
gc = GPConnector()

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from openpyxl import load_workbook
from openpyxl.styles import Alignment,Border,Side,PatternFill
pd.set_option('display.max_colwidth',255)

### Формирование таблицы выставлений

In [61]:
gc.execute('''
delete from s_grnplm_ld_cib_sbc_custtmp.dich_novreg_vist_report
where dt::date =  current_date::date - 31;

insert into s_grnplm_ld_cib_sbc_custtmp.dich_novreg_vist_report
with start_inns as (
    select a.cust_inn                                                                          as inn
         , regexp_replace(reg_match_contact, '[^0-9]', '')                                     as phone
         , case when b.PERSON_FULL_NM is not null then b.PERSON_FULL_NM else 'Отсутствует' end as FIO
         , c.REG_MATCH_CONTACT_MAIN_FLG
         , reg_match_contact
         , a.REG_EVENT_DTTM
         , person_inn
         , a.cust_full_nm
         , b.reg_match_ucprb_person_ref                                                        as epk_id
         , REG_EVENT_TYPE_NM
         , REG_MATCH_REFUSAL_FLG
         , cust_inn
    from s_grnplm_ld_cib_sbc_custref.v_customer_register_event_mig a
             left join
         s_grnplm_ld_cib_sbc_custref.v_customer_register_match_mig b
         on a.reg_event_hash = b.reg_event_hash
             left join
         s_grnplm_ld_cib_sbc_custref.v_customer_register_contact_mig c
         on b.reg_match_hash = c.reg_match_hash
             and REG_MATCH_CONTACT_TYPE_NM = 'Телефон'

    where a.REG_EVENT_DTTM::date = current_date - case extract(DOW from current_date)
                                                      when 1 then 33
                                                      else 31 end
    --       and a.REG_EVENT_TYPE_NM = 'Регистрация ЮЛ/ИП'
--       and REG_MATCH_REFUSAL_FLG = 0
--       and c.reg_match_contact is not null
--       and b.person_inn is not null
--       and a.cust_inn is not null
--       and substr(regexp_replace(reg_match_contact, '[^0-9]', ''), 1, 1) in ('3', '4', '8', '9')
--       and length(regexp_replace(reg_match_contact, '[^0-9]', '')) = 10
--       and not (a.cust_inn like '91%' or a.cust_inn like '92%' or a.cust_inn like '93%' or a.cust_inn like '94%' or
--                a.cust_inn like '95%' or a.cust_inn like '90%')
)
   , SHILU_PRIVLNOVREG_CRIMEA as (
    select a.cust_inn                                                                          as inn
         , regexp_replace(reg_match_contact, '[^0-9]', '')                                     as phone
         , case when b.PERSON_FULL_NM is not null then b.PERSON_FULL_NM else 'Отсутствует' end as FIO
         , c.REG_MATCH_CONTACT_MAIN_FLG
         , reg_match_contact
         , a.REG_EVENT_DTTM
         , person_inn
         , a.cust_full_nm
         , b.reg_match_ucprb_person_ref                                                        as epk_id
         , REG_EVENT_TYPE_NM
         , REG_MATCH_REFUSAL_FLG
         , cust_inn
    from s_grnplm_ld_cib_sbc_custref.v_customer_register_event_mig a
             left join
         s_grnplm_ld_cib_sbc_custref.v_customer_register_match_mig b
         on a.reg_event_hash = b.reg_event_hash
             left join
         s_grnplm_ld_cib_sbc_custref.v_customer_register_contact_mig c
         on b.reg_match_hash = c.reg_match_hash
             and REG_MATCH_CONTACT_TYPE_NM = 'Телефон'

    where a.REG_EVENT_DTTM::date = current_date - case extract(DOW from current_date)
                                                      when 1 then 33
                                                      else 31 end
      and a.REG_EVENT_TYPE_NM = 'Регистрация ЮЛ/ИП'
      and REG_MATCH_REFUSAL_FLG = 0
      and c.reg_match_contact is not null
      and b.person_inn is not null
      and a.cust_inn is not null
      and substr(regexp_replace(reg_match_contact, '[^0-9]', ''), 1, 1) in ('3', '4', '8', '9')
      and length(regexp_replace(reg_match_contact, '[^0-9]', '')) = 10
      and not (a.cust_inn like '91%' or a.cust_inn like '92%' or a.cust_inn like '93%' or a.cust_inn like '94%' or
               a.cust_inn like '95%' or a.cust_inn like '90%')
)
   , pre_2 as (
    select a.*
         , row_number() over (partition by a.inn order by a.inn,REG_MATCH_CONTACT_MAIN_FLG desc) as rn_inn
    from SHILU_PRIVLNOVREG_CRIMEA a
)
   , novreg_crim as (
    select a.inn
         , phone
         , FIO
         , REG_EVENT_DTTM
         , person_inn
         , cust_full_nm
         , epk_id
    from pre_2 a left join s_grnplm_ld_cib_sbc_custref.v_ma_unified_customer b on a.cust_inn = b.inn
    where rn_inn = 1
      and (coalesce(okato_lv2_cd, '0') not in ('21401', '43401') and
           not (a.cust_inn like '91%' or a.cust_inn like '92%' or a.cust_inn like '93%' or a.cust_inn like '94%' or
                a.cust_inn like '95%' or a.cust_inn like '90%')
        and coalesce(substr(okato_cd, 1, 2), '0') not in ('31', '48', '46')
        )
)
   , po_pre as (
    select cust_inn                   as inn
         , case
               when offer_status_cd in
                    ('NEW_OFFER', 'IN_PROGRESS', 'CLIENT_THINK', 'PROPOSAL_PREPARING', 'PROPOSAL_APPROVING')
                   then create_dt::date + 10
               when offer_status_cd in
                    ('INTEREST_CONFIRMED', 'DEAL_CREATED', 'PRODUCT_IS_NOT_INTERESTING', 'CLOSED',
                     'EXPANSION_WITHOUT_DEAL', 'ON_APPROVAL', 'PROPOSAL_AGREED', 'APPLICATION_SIGNED')
                   and coalesce(offer_refusal_cd, '0') not in
                       ('UNCORRECT_OFFER', 'CANCELLED', 'MISSED_CALL', 'OUTDATED_CONTACTS', 'DEACTIVATED_IN_SAS',
                        'COMMUNICATION_NOT_HAPPENED', 'OFFER_NOT_COMPLETED', 'OFFER_CREATION_FAILURE',
                        'NOT_DISCUSSED', 'LACK_OF_CONTACTS', 'CENTRALIZED_CANCELLED', 'USER_CANCELLED',
                        'NOT_DISCUSSED', 'EXPIRED', 'IS_NOT_LPR', 'REFUSAL_TO_TALK', 'REFUSAL_OF_COMMUNICATION')
                   then update_dt end as kp_date
    from s_grnplm_ld_cib_sbc_custref.v_comm_offer_mig
    where 1 = 1
      and offer_recommendation_flg <> 1
      and offer_src_cd not in
          ('SBER_BUSINESS', 'SBBOL', 'EFS_MZP_RB', 'SBBOL PPRB', 'KFUL', 'SITE', 'EXPERT', 'VED', 'AST')
      and offer_stage_cd in ('SELLING', 'EXPANSION_OF_COOPERATION', 'CHANGING_CONDITIONS')
      and create_dt >= current_date - 365
)
   , po as (
    select inn, coalesce(kp_date, TO_DATE('2007-04-02', 'YYYY-MM-DD')) as dt
    from po_pre
    where coalesce(kp_date, TO_DATE('2007-04-02', 'YYYY-MM-DD')) >= current_date - 7
)
   , task_pre as (
    select cust_inn as inn
         , case
               when task_status_nm in ('NEW', 'PLANNED', 'IN_PROGRESS') then
                   case
                       when task_type_cd in
                            ('OUTFLOW', 'ONBOARDING', 'FOLLOWUP', 'MEET_MANAGER', 'RENEWAL_OF_COOPERATION',
                             'CONGRATULATION_BIRTHDAY', 'SERVICE_TASK_ORGANIZATION', 'CUSTOMER_SERVICE',
                             'ACTIVATION', 'SURVEY', 'NEWS', 'CONGRATULATION', 'PRODUCT_REGISTRATION') and
                            task_category_cd = 'TASK'
                           then create_dt::date + 10
                       end
               when task_status_nm in ('DONE', 'CANCELLED')
                   and coalesce(task_result_cd, '0') not in
                       ('COMMUNICATION_NOT_HAPPENED', 'NOT_APPROVED', 'IS_NOT_LPR', 'COMPLIANCE_BLOKING',
                        'CLOSED_SET_INCORRECTLY',
                        'EXPIRED', 'NOT_AN_URGENT_TASK', 'NOT_ACCOMPLISHED', 'NOT_DISCUSSED',
                        'FAILED_TO_RESOLVED_THE_ISSUE', 'NOT_PHONED', 'INVALID_CONTACT', 'INCORRECT_CONTACTS',
                        'USER_CANCELLED', 'LACK_OF_CONTACTS', 'MOVED', 'CENTRALIZED_CANCELLED', 'NOT_RESPOND',
                        'CUSTOMER_IS_NOT_INFORMED', 'REFUSAL_TO_TALK', 'REFUSAL_OF_COMMUNICATION')
                   then
                   case
                       when task_type_cd in
                            ('OUTFLOW', 'ONBOARDING', 'FOLLOWUP', 'MEET_MANAGER', 'RENEWAL_OF_COOPERATION',
                             'CONGRATULATION_BIRTHDAY', 'SERVICE_TASK_ORGANIZATION', 'CUSTOMER_SERVICE',
                             'ACTIVATION', 'SURVEY', 'NEWS', 'CONGRATULATION', 'PRODUCT_REGISTRATION') and
                            task_category_cd = 'TASK'
                           then coalesce(task_end_fact_dt::date, update_dt::date)
                       end
        end         as kp_date
    from s_grnplm_ld_cib_sbc_custref.v_comm_task_mig
    where 1 = 1
      and task_recomendation_flg <> 1
      and task_src_cd not in
          ('SBER_BUSINESS', 'SBBOL', 'EFS_MZP_RB', 'SBBOL PPRB', 'KFUL', 'SITE', 'EXPERT', 'VED',
           'AST')
      and create_dt >= current_date - 365
      and task_type_cd in
          ('CALLING', 'APPOINTMENT', 'OUTFLOW', 'ONBOARDING', 'FOLLOWUP', 'MEET_MANAGER', 'RENEWAL_OF_COOPERATION',
           'CONGRATULATION_BIRTHDAY', 'SERVICE_TASK_ORGANIZATION', 'CUSTOMER_SERVICE', 'ACTIVATION', 'SURVEY',
           'NEWS', 'CONGRATULATION', 'PRODUCT_REGISTRATION')
)
   , task as (
    select inn, coalesce(kp_date, TO_DATE('2007-04-02', 'YYYY-MM-DD')) as dt
    from task_pre
    where coalesce(kp_date, TO_DATE('2007-04-02', 'YYYY-MM-DD')) >= current_date - 7
)
   , icc as (
    select inn
    from s_grnplm_ld_cib_sbc_custref.v_cor_insight
    where load_dt >= current_date - 7
      and target_param_cd in (select target_param_cd
                              from s_grnplm_ld_cib_sbc_custref.v_target_param
                              where not (
                                          target_param_nm ilike '%ЦА-ПРИВЛТМФАФНС%'
                                      or target_param_nm ilike '%ЦА-ПРИВЛТМФАКЭПФЛ%'
                                      or target_param_nm ilike '%ЦА-ПРИВЛТМФАКЭП%'
                                      or target_param_nm ilike '%ЦА-ПРИВЛТМФАИНИ%'
                                      or target_param_nm ilike '%ЦА-ПРИВЛТМФАДОС%'
                                      or target_param_nm ilike '%ЦА-ПРИВЛТМФАПРЕДРЕГ%'
                                      or target_param_nm ilike '%ЦА-ПРИВЛТМФАОНЛАЙН%'
                                      or target_param_nm ilike '%ЦА-ПРИВЛГП%'
                                      or target_param_nm ilike '%ЦА-ПРИВЛНОВРЕГМБА%'
                                  )
                                and channel_group_cd = 'MMB_TM')
    union
    select cust_inn as inn
    from s_grnplm_ld_cib_sbc_custref.v_comm_task_mig
    where create_dt >= current_date - 7
)
   , client as (select cust_inn as inn
                from s_grnplm_ld_cib_sbc_custref.v_agreement_mig
                where prod_group_nm = 'Договор РКО'
                  and agr_end_fact_dt is null
                  and cust_inn is not null)
   , bank as (
    select trim(concat(person_last_nm, ' ', person_first_nm, ' ', person_middle_nm)) as fio
    from s_grnplm_ld_cib_sbc_custref.v_employee_mig t1
    where (emp_position_nm ilike '%президент%'
        or emp_position_nm ilike '%председ%'
        or emp_position_nm ilike '%советник%')

      and not (emp_position_nm ilike '%исполнительны%'
        or emp_position_nm ilike '%офиса%'
        or emp_position_nm ilike '%всп%'
        or emp_position_nm ilike 'Руководитель направления%'
        )
      and emp_position_end_dt is null
)

   , pre_fin as (
    select a.inn                 as cust_inn
         , a.phone
         , a.fio
         , a.person_inn          as inn
         , substr(a.phone, 1, 1) as substr_phone
         , length(a.phone)       as ph_len
         , cust_full_nm
    from novreg_crim a
    where 7 = 7
      and not (a.inn like '91%' or a.inn like '92%' or a.inn like '93%' or a.inn like '94%' or
               a.inn like '95%' or a.inn like '90%')
      and a.phone is not null
      and a.fio is not null)

   , fin as (
    select distinct cust_inn
                  , phone
                  , fio
                  , inn
                  , cust_full_nm
    from pre_fin
    where 7 = 7
      and substr_phone in ('3', '4', '8', '9')
      and ph_len = 10
)
   , pre as (
    select a.cust_inn
         , a.phone
         , a.fio
         , a.inn
         , cust_full_nm
         ,case when t2.inn is null then 0 else 1 end as     task_7_flg
         ,case when  icc.inn is null then 0 else 1 end as   tm_7_flg
         ,case when po1.inn is null then 0 else 1 end as    offer_1_flg
         ,case when agr.inn is null then 0 else 1 end as    have_rko_flg
         ,case when bank.fio is null then 0 else 1 end as   top_fio_flg
    from fin a
             left join
         task t2
         on a.cust_inn = t2.inn
             left join
         icc
         on a.cust_inn = icc.inn
             left join
         po po1
         on a.cust_inn = po1.inn
             left join
         client agr
         on a.cust_inn = agr.inn
             left join
         bank
         on lower(a.fio) = lower(bank.fio)
    where 1 = 1
)
select (current_timestamp - interval '31 days')                                              as dt

     , (select count(distinct cust_inn) from start_inns) as start

     , (select count(distinct inn)
        from start_inns
        where 7 = 7
          and REG_EVENT_TYPE_NM = 'Регистрация ЮЛ/ИП')                 as ip_ul

     , (select count(distinct inn)
        from start_inns
        where 7 = 7
          and REG_EVENT_TYPE_NM = 'Регистрация ЮЛ/ИП'
          and REG_MATCH_REFUSAL_FLG = 0)                               as rej_flg

     , (select count(distinct inn)
        from start_inns
        where 7 = 7
          and REG_EVENT_TYPE_NM = 'Регистрация ЮЛ/ИП'
          and REG_MATCH_REFUSAL_FLG = 0
          and reg_match_contact is not null)                           as have_phone

     , (select count(distinct inn)
        from start_inns
        where 7 = 7
          and REG_EVENT_TYPE_NM = 'Регистрация ЮЛ/ИП'
          and REG_MATCH_REFUSAL_FLG = 0
          and reg_match_contact is not null
          and person_inn is not null)                                  as have_fl_flg

     , (select count(distinct inn)
        from start_inns
        where 7 = 7
          and REG_EVENT_TYPE_NM = 'Регистрация ЮЛ/ИП'
          and REG_MATCH_REFUSAL_FLG = 0
          and reg_match_contact is not null
          and person_inn is not null
          and cust_inn is not null)                                    as have_inn_ul

     , (select count(distinct inn)
        from start_inns
        where 7 = 7
          and REG_EVENT_TYPE_NM = 'Регистрация ЮЛ/ИП'
          and REG_MATCH_REFUSAL_FLG = 0
          and reg_match_contact is not null
          and person_inn is not null
          and cust_inn is not null
          and substr(regexp_replace(reg_match_contact, '[^0-9]', ''), 1, 1) in ('3', '4', '8', '9')
          and length(regexp_replace(reg_match_contact, '[^0-9]', '')) = 10
)                                                                      as val_phone_flg


     , (select count(distinct a.cust_inn)
        from start_inns a left join s_grnplm_ld_cib_sbc_custref.v_ma_unified_customer b on a.cust_inn = b.inn
        where 7 = 7
          and REG_EVENT_TYPE_NM = 'Регистрация ЮЛ/ИП'
          and REG_MATCH_REFUSAL_FLG = 0
          and reg_match_contact is not null
          and person_inn is not null
          and cust_inn is not null
          and substr(regexp_replace(reg_match_contact, '[^0-9]', ''), 1, 1) in ('3', '4', '8', '9')
          and length(regexp_replace(reg_match_contact, '[^0-9]', '')) = 10
          and (coalesce(okato_lv2_cd, '0') not in ('21401', '43401') and
               not (a.cust_inn like '91%' or a.cust_inn like '92%' or a.cust_inn like '93%' or a.cust_inn like '94%' or
                    a.cust_inn like '95%' or a.cust_inn like '90%')
            and coalesce(substr(okato_cd,1,2),'0') not in ('31','48','46')
            )
)                                                                      as no_new_ter

     ,(select count(distinct inn) from novreg_crim) as main_contact_flg
     ,(select count(distinct cust_inn) from fin) as val_main_phone
     ,(select count(distinct cust_inn) from pre where 7=7 and task_7_flg = 0) as task_7
     ,(select count(distinct cust_inn) from pre where 7=7 and task_7_flg = 0 and tm_7_flg = 0) as tm_7
     ,(select count(distinct cust_inn) from pre where 7=7 and task_7_flg = 0 and tm_7_flg = 0 and offer_1_flg = 0) as offer_7
     ,(select count(distinct cust_inn) from pre where 7=7 and task_7_flg = 0 and tm_7_flg = 0 and offer_1_flg = 0 and have_rko_flg = 0) as have_rko
     ,(select count(distinct cust_inn) from pre where 7=7 and task_7_flg = 0 and tm_7_flg = 0 and offer_1_flg = 0 and have_rko_flg = 0 and top_fio_flg = 0) as top_fio
     ,(select count(distinct t1.inn)
       from s_grnplm_ld_cib_sbc_custref.v_insight_factory t1 join pre t2 on t1.inn = t2.cust_inn and task_7_flg = 0 and tm_7_flg = 0 and offer_1_flg = 0 and have_rko_flg = 0 and top_fio_flg = 0
       where target_launch_id in (
           select target_launch_id
           from s_grnplm_ld_cib_sbc_custref.v_target_launch
           where target_param_cd in ('T02227_05_24','T03402_08_24')
       )) as block_stops
     ,(select count(distinct t3.inn)
       from s_grnplm_ld_cib_sbc_custref.v_sbc_adapter_sbercrm_svc_contact_history t1
                join s_grnplm_ld_cib_sbc_custref.v_sbc_adapter_sbercrm_svc_contact_status t2
                     on t1.contact_id = t2.contact_id
                join s_grnplm_ld_cib_sbc_custref.v_ma_unified_customer t3
                     on t1.epk_id::text = t3.mdm_id::text
                join pre t4 on t3.inn = t4.cust_inn and task_7_flg = 0 and tm_7_flg = 0 and offer_1_flg = 0 and have_rko_flg = 0 and top_fio_flg = 0
       where campaign_name in ('SL1150-05-24-01','SL1150-05-24-02')
         and t1.created::date = current_date) as after_sbc_checks
''')

### Формирование таблицы динамики регистраций

In [62]:
gc.execute('''
drop table if exists s_grnplm_ld_cib_sbc_custtmp.dich_nr_report;
create table s_grnplm_ld_cib_sbc_custtmp.dich_nr_report as
with all_dates as (select cust_inn,reg_event_dttm::date as reg_event_dttm
                   from s_grnplm_ld_cib_sbc_custref.v_customer_register_event_mig
),
     ip as (
         SELECT count(DISTINCT cust_inn) AS inn,reg_event_dttm::date as reg_event_dttm
         FROM s_grnplm_ld_cib_sbc_custref.v_customer_register_event_mig
         WHERE 7=7
           AND REG_EVENT_SOURCE_NM IN ('ЕГРИП')
         group by reg_event_dttm::date
     ),
     ul as (
         SELECT count(DISTINCT cust_inn) AS inn,reg_event_dttm::date as reg_event_dttm
         FROM s_grnplm_ld_cib_sbc_custref.v_customer_register_event_mig
         WHERE 7=7
           AND REG_EVENT_SOURCE_NM IN ('ЕГРЮЛ')
         group by reg_event_dttm::date
     ),
     inn_avail as (
         SELECT count(DISTINCT cust_inn) AS inn,reg_event_dttm::date as reg_event_dttm
         FROM s_grnplm_ld_cib_sbc_custref.v_customer_register_event_mig a
                  left join
              s_grnplm_ld_cib_sbc_custref.v_customer_register_match_mig b
              on a.reg_event_hash = b.reg_event_hash
                  left join
              s_grnplm_ld_cib_sbc_custref.v_customer_register_contact_mig c
              on b.reg_match_hash = c.reg_match_hash
         where  REG_EVENT_SOURCE_NM IN ('ЕГРЮЛ','ЕГРИП')
           and reg_match_contact is not null
         group by reg_event_dttm::date
     ),
     inn_avail_no_crimea as (
         SELECT count(DISTINCT cust_inn) AS inn,reg_event_dttm::date as reg_event_dttm
         FROM s_grnplm_ld_cib_sbc_custref.v_customer_register_event_mig a
                  left join
              s_grnplm_ld_cib_sbc_custref.v_customer_register_match_mig b
              on a.reg_event_hash = b.reg_event_hash
                  left join
              s_grnplm_ld_cib_sbc_custref.v_customer_register_contact_mig c
              on b.reg_match_hash = c.reg_match_hash
         where 7=7
           AND REG_EVENT_SOURCE_NM IN ('ЕГРЮЛ','ЕГРИП')
           AND cust_inn !~ '^(90|91|92|93|94|95)'
           and reg_match_contact is not null
         group by reg_event_dttm::date
     )
--         ,have_phone as (
--     SELECT count(DISTINCT cust_inn) AS inn,reg_event_dttm::date as reg_event_dttm
--     from    s_grnplm_ld_cib_sbc_custref.v_customer_register_event_mig a
--                 left join
--             s_grnplm_ld_cib_sbc_custref.v_customer_register_match_mig b
--             on a.reg_event_hash = b.reg_event_hash
--                 left join
--             s_grnplm_ld_cib_sbc_custref.v_customer_register_contact_mig c
--             on b.reg_match_hash = c.reg_match_hash
--     WHERE 7=7
--       AND REG_EVENT_SOURCE_NM IN ('ЕГРЮЛ','ЕГРИП')
--       AND cust_inn !~ '^(90|91|92|93|94|95)'
--       and a.REG_EVENT_TYPE_NM = 'Регистрация ЮЛ/ИП'
--       and substr(regexp_replace(reg_match_contact, '[^0-9]', ''), 1, 1) in ('3', '4', '8', '9')
--       and length(regexp_replace(reg_match_contact, '[^0-9]', '')) = 10
--       and REG_MATCH_CONTACT_TYPE_NM = 'Телефон'
--     group by reg_event_dttm::date
-- )
        ,pk as (
    select cust_inn ,create_dt::date as dt
    from s_grnplm_ld_cib_sbc_custref.v_comm_task_mig
    union
    select cust_inn ,create_dt::date as dt
    from s_grnplm_ld_cib_sbc_custref.v_comm_offer_mig
),
     pk_2 as (
         select count(distinct t1.cust_inn) as cnt,reg_event_dttm::date
         from all_dates t1
                  inner join pk t2 on t1.cust_inn = t2.cust_inn
             and t2.dt between t1.reg_event_dttm - 7 and t1.reg_event_dttm
         group by t1.reg_event_dttm::date
     )
        ,vist as (
    select inn
    from s_grnplm_ld_cib_sbc_custref.v_ma_icc_history
    where campaign_cd in ('CAMP24456','CAMP24682')
    union
    select cust_inn as inn
    from s_grnplm_ld_cib_sbc_custref.v_comm_task_mig
    where task_campaign_cd in ('SL1150-05-24-01','SL1150-05-24-02')
)
        ,vist_dates as (
    select count(distinct t1.cust_inn) as cnt,least(coalesce(registration_dt,'9999-01-01'::date),reg_event_dttm::date)::date as reg_event_dttm
    from all_dates t1
             inner join vist t2 on t1.cust_inn = t2.inn
             left join s_grnplm_ld_cib_sbc_custref.v_ma_unified_customer vmuc
                       on t1.cust_inn = vmuc.inn
    group by least(coalesce(registration_dt,'9999-01-01'::date),reg_event_dttm::date)
)
select distinct t1.reg_event_dttm as date_reg
              ,ip.inn ip
              ,ul.inn ul
              ,coalesce(ip.inn,0) + coalesce(ul.inn,0) as  ip_ul
              ,inn_avail.inn as  inn_avail
              ,inn_avail_no_crimea.inn as no_crimea_inn
--               ,case when (coalesce(ip.inn,0) + coalesce(ul.inn,0)) = 0 then '0' else
--         round(inn_avail.inn::numeric / (coalesce(ip.inn,0) + coalesce(ul.inn,0))::numeric * 100) || '%' end as have_phone
              ,pk_2.cnt as pk
              ,round((pk_2.cnt::numeric / (coalesce(ip.inn,0) + ul.inn)::numeric) * 100,1) || '%' as pk_perc
--               ,case when vist_dates.cnt > 10 then vist_dates.cnt else 0 end as vist
--               ,round((case when vist_dates.cnt > 10 then vist_dates.cnt else 0 end)::numeric / inn_avail.inn::numeric * 100) || '%' as ohv
from all_dates t1
         left join ip on t1.reg_event_dttm = ip.reg_event_dttm
         left join ul on t1.reg_event_dttm = ul.reg_event_dttm
         left join inn_avail on t1.reg_event_dttm = inn_avail.reg_event_dttm
         left join inn_avail_no_crimea on t1.reg_event_dttm = inn_avail_no_crimea.reg_event_dttm
--          left join have_phone on t1.reg_event_dttm = have_phone.reg_event_dttm
         left join pk_2 on t1.reg_event_dttm = pk_2.reg_event_dttm
         left join vist_dates on t1.reg_event_dttm = vist_dates.reg_event_dttm
         left join s_grnplm_ld_cib_sbc_custref.v_dict_production_calendar_copy_cjm cal
                   on t1.reg_event_dttm = cal.day
                       and is_work_day = 'false'
where cal.day is null
order by 1 desc
''')

### Подготовка датафреймов

In [63]:
#Динамика регистраций ежедневно
df = gc.select('''
select
date_reg
,ip
,ul
,ip_ul
,inn_avail
,no_crimea_inn
,pk
,pk_perc

from
s_grnplm_ld_cib_sbc_custtmp.dich_nr_report
order by 1 desc
limit 60
''')

table = df.copy(deep=True)
table = table.fillna(0)
table.date_reg = pd.to_datetime(table.date_reg)
table.ip = table.ip.astype('int32')
table.ul = table.ul.astype('int32')
table.ip_ul = table.ip_ul.astype('int32')
table.inn_avail = table.inn_avail.astype('int32')
table.sort_values(by='date_reg',ascending=False,inplace=True)
table['date_reg'] = table['date_reg'].dt.strftime('%d.%m.%Y')

In [64]:
##Динамика регистраций ежемесячно
df_2 = gc.select('''
select to_char(date_reg::date,'YYYY-MM') as mont
     ,round(sum(ip)/1000) as ip
     ,round(sum(ul)/1000) as ul
     ,round(sum(ip_ul)/1000) as ip_ul
     ,round(sum(inn_avail)/1000) as inn_avail
     ,sum(pk) as pk
     ,round((sum(pk) / sum(ip_ul)) * 100) || '%' as pk_perc
     ,round((sum(no_crimea_inn) / sum(inn_avail)) * 100) || '%' as have_phone_perc
     ,round(likv_cnt/1000) as likv_cnt

from s_grnplm_ld_cib_sbc_custtmp.dich_nr_report t1
left join (select count(distinct inn) as likv_cnt,to_char(status_dt::date,'YYYY-MM') as mon_likv
           from s_grnplm_ld_cib_sbc_custref.v_ma_unified_customer
           where active_flg = 0
           group by to_char(status_dt::date,'YYYY-MM')) t2
on to_char(date_reg::date,'YYYY-MM') = t2.mon_likv
group by to_char(date_reg::date,'YYYY-MM'),likv_cnt
''')

table_2 = df_2.copy(deep=True)
table_2 = table_2.fillna(0)
table_2.mont = pd.to_datetime(table_2.mont)
table_2.ip = table_2.ip.astype('int32')
table_2.ul = table_2.ul.astype('int32')
table_2.ip_ul = table_2.ip_ul.astype('int32')
table_2.likv_cnt = table_2.likv_cnt.astype('int32')
table_2.sort_values(by='mont',ascending=False,inplace=True)
table_2['mont'] = table_2['mont'].dt.strftime('%Y-%m')

In [65]:
#Воронка отсечений
df_3 = gc.select('''
select dt::date as dt,
       --start,
       ip_ul,
       rej_flg,
       have_phone,
       have_fl_flg,
       have_inn_ul,
       val_phone_flg,
       no_new_ter,
       offer_7,
       have_rko,
       top_fio,
       block_stops,
       after_sbc_checks
from s_grnplm_ld_cib_sbc_custtmp.dich_novreg_vist_report
''')

table_3 = df_3.copy(deep=True)
table_3 = table_3.fillna(0)
table_3.dt = pd.to_datetime(table_3.dt)
table_3.ip_ul = table_3.ip_ul.astype('int32')
table_3.rej_flg = table_3.rej_flg.astype('int32')
table_3.have_phone = table_3.have_phone.astype('int32')
table_3.have_fl_flg = table_3.have_fl_flg.astype('int32')
table_3.have_inn_ul = table_3.have_inn_ul.astype('int32')
table_3.val_phone_flg = table_3.val_phone_flg.astype('int32')
table_3.no_new_ter = table_3.no_new_ter.astype('int32')
table_3.offer_7 = table_3.offer_7.astype('int32')
table_3.have_rko = table_3.have_rko.astype('int32')
table_3.top_fio = table_3.top_fio.astype('int32')
table_3.block_stops = table_3.block_stops.astype('int32')
table_3.after_sbc_checks = table_3.after_sbc_checks.astype('int32')
table_3['dt'] = table_3['dt'].dt.strftime('%d.%m.%Y')
table_3.sort_values             (by='dt',ascending=False,inplace=True)
table_3

,dt,ip_ul,rej_flg,have_phone,have_fl_flg,have_inn_ul,val_phone_flg,no_new_ter,offer_7,have_rko,top_fio,block_stops,after_sbc_checks
4,30.07.2024,3058,2267,2267,2267,2267,2267,2051,1794,1260,1257,1249,1018
2,29.07.2024,1105,802,801,801,801,801,797,742,529,529,527,519
3,28.07.2024,6,5,5,5,5,5,5,5,3,3,3,3
6,27.07.2024,423,314,314,314,314,314,279,245,178,178,177,173
1,26.07.2024,5058,3656,3656,3656,3656,3655,3411,2995,2319,2319,2293,2241
0,23.07.2024,3963,2915,2912,2912,2912,2912,2572,2268,1756,1755,1741,1671
5,22.07.2024,3776,2762,2761,2761,2761,2761,2524,2200,1632,1632,1627,1575


In [66]:
#Стабильность передачи базы
df4 = gc.select('''
with vist as (select  max(t1.created) as dt_start,t1.created::date as dt
              from s_grnplm_ld_cib_sbc_custref.v_sbc_adapter_sbercrm_svc_contact_history t1
                       join s_grnplm_ld_cib_sbc_custref.v_sbc_adapter_sbercrm_svc_contact_status t2
                            on t1.contact_id = t2.contact_id
              where campaign_name in ('SL1150-05-24-01', 'SL1150-05-24-02')
              group by t1.created::date
),
     zapusk as (
         select max(launch_dttm) as dt_start,launch_dttm::date as dt
         from s_grnplm_ld_cib_sbc_custref.v_target_launch
         where target_param_cd in ('T02227_05_24','T03402_08_24')
         group by launch_dttm::date
     )

select distinct
      t1.dt
,substr(t1.dt_start::text,12,8) as launch_dt
,substr(t2.dt_start::text,12,8) as vist_dt
,case when extract(hour from t2.dt_start)::numeric < 9 then 'Да' else 'Нет' end as stable_flg
from zapusk t1 left join vist t2
on t1.dt = t2.dt
order by t1.dt desc
limit 20
''')

table_4 = df4.copy(deep=True)
table_4.dt = pd.to_datetime(table_4.dt)
table_4.sort_values (by='dt',ascending=False,inplace=True)
table_4['dt'] = table_4['dt'].dt.strftime('%d.%m.%Y')
table_4

,dt,launch_dt,vist_dt,stable_flg
0,30.08.2024,07:00:11,07:09:36,Да
1,29.08.2024,07:00:10,07:10:31,Да
2,28.08.2024,07:00:08,07:10:30,Да
3,27.08.2024,13:49:16,13:57:39,Нет
4,26.08.2024,07:00:07,07:06:31,Да
5,23.08.2024,07:00:10,07:09:29,Да
6,22.08.2024,07:16:10,07:27:26,Да
7,21.08.2024,07:00:29,07:52:46,Да
8,20.08.2024,07:00:11,09:33:52,Нет
9,19.08.2024,07:00:08,07:07:50,Да


### Формирование финального экселя

In [68]:
with pd.ExcelWriter('excl.xlsx', engine='xlsxwriter') as writer:
    #--------------------------------------------------------------------------------------------------------------------------------------------------------------
    table.to_excel(writer, index=False, startrow=1, sheet_name='Ежедневно_Кампании',header=[
        '"Дата регистрации"'
        ,'"Кол-во рег. ИП"'
        ,'"Кол-во рег. ЮЛ"'
        ,'"ИП+ЮЛ"'
        ,'"Кол-во доступных к привлечению (ИП+ЮЛ)"'
        ,'"Кол-во доступных к привлечению (ИП+ЮЛ) без новых территорий"'
        ,'"Кол-во лидов партнеров"'
        ,"% клиентов, отработанных партнерами"
        # ,'"Переданы в рамках кампании Ежедневно_Кампании"'
        # ,'"% Охвата кампаний продаж"'
    ])
    #--------------------------------------------------------------------------------------------------------------------------------------------------------------
    table_2.to_excel(writer, index=False, startrow=1, sheet_name='Ежемесячно_Кампании',header=[
        '"Отчетный месяц"'
        ,'"Кол-во рег. ИП" тыс.'
        ,'"Кол-во рег. ЮЛ" тыс.'
        ,'"ИП+ЮЛ" тыс.'
        ,'"Кол-во доступных к привлечению (ИП+ЮЛ)" тыс.'
        ,'"Кол-во лидов партнеров"'
        ,'% клиентов, отработанных партнерами"'
        ,'"Кол-во доступных к привлечению (ИП+ЮЛ) без новых территорий"'
        ,'"Количество ликвидированных организаций", тыс.'
        # ,'% охвата кампаний продаж по Новому бизнесу'
    ])
    #--------------------------------------------------------------------------------------------------------------------------------------------------------------
    table_3.to_excel(writer, index=False, startrow=1, sheet_name='Воронка отсечений',header=[
        '"Дата регистрации"'
        ,'"Кол-во новых регистраций'
        ,'"Кол-во новорегов без отказа от коммуникации'
        ,'"Кол-во новорегов с номером телефона'
        ,'Кол-во новорегов с не пустым ФЛ ИНН'
        ,'Кол-во новорегов с не пустым ЮЛ ИНН'
        ,'Кол-во новорегов с валидным номером телефона'
        ,'Кол-во новорегов без новых территорий'
        ,'Кол-во новорегов, прошедших конт. политику 7 дней'
        ,'Кол-во новорегов, без РКО в Сбер'
        ,'Исключение ТОП-ов Сбера'
        ,'Кол-во новорегов, прошедших блокирующие стопы'
        ,'Кол-во новорегов, прошедших внутренние SBC проверки'
    ])
    #--------------------------------------------------------------------------------------------------------------------------------------------------------------
    table_4.to_excel(writer, index=False, startrow=1, sheet_name='Стабильность передачи базы',header=[
        'Дата запуска'
        ,'Время запуска'
        ,'Время выставления в канал'
        ,'Поставка до 9:00 МСК'
    ])
    #--------------------------------------------------------------------------------------------------------------------------------------------------------------
    workbook = writer.book

    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'vcenter',
        'font_size': 14,
        'align': 'center'
    })
    cell_format = workbook.add_format({
        'text_wrap': True,
        'valign': 'vcenter',
        'align': 'center'
    })
    #--------------------------------------------------------------------------------------------------------------------------------------------------------------
    worksheet = writer.sheets['Ежедневно_Кампании']
    worksheet.merge_range('A1:H1', 'Динамика рынка новых регистраций (ИП+ЮЛ) и охвата кампаний продаж по ним', header_format)

    for idx, col in enumerate(table.columns):  # Excel columns start from 0
       worksheet.set_column(idx, idx, 20, cell_format)  # Adjust the column width and apply format

    #--------------------------------------------------------------------------------------------------------------------------------------------------------------
    worksheet = writer.sheets['Ежемесячно_Кампании']

    worksheet.merge_range('A1:I1', 'Динамика кампании Ежедневно_Кампании', header_format)

    for idx, col in enumerate(table_2.columns):  # Excel columns start from 0
       worksheet.set_column(idx, idx, 20, cell_format)  # Adjust the column width and apply format
    #--------------------------------------------------------------------------------------------------------------------------------------------------------------
    worksheet = writer.sheets['Воронка отсечений']

    worksheet.merge_range('A1:M1', 'Воронка отсечений', header_format)

    for idx, col in enumerate(table_3.columns):  # Excel columns start from 0
        worksheet.set_column(idx, idx, 20, cell_format)  # Adjust the column width and apply format
    #--------------------------------------------------------------------------------------------------------------------------------------------------------------
    worksheet = writer.sheets['Стабильность передачи базы']

    worksheet.merge_range('A1:D1', 'Новые регистрации', header_format)

    for idx, col in enumerate(table_4.columns):  # Excel columns start from 0
        worksheet.set_column(idx, idx, 20, cell_format)  # Adjust the column width and apply format



#--------------------------------------------------------------------------------------------------------------------------------------------------------------
wb = load_workbook('excl.xlsx')
#--------------------------------------------------------------------------------------------------------------------------------------------------------------
sheet = wb['Ежедневно_Кампании']

for i in ['A2','B2','C2','D2','E2','F2','G2','H2']:
    cell = sheet[i]

    cell.alignment = Alignment(wrapText=True,horizontal='center',vertical='center')


thin_border = Border(left=Side(style='thin',color='006400'),
                     right=Side(style='thin',color='006400'),
                     top=Side(style='thin',color='006400'),
                     bottom=Side(style='thin',color='006400'))


for row in range(2, sheet.max_row + 1):
    for col in range(1, sheet.max_column + 1):
        sheet.cell(row=row, column=col).border = thin_border

#--------------------------------------------------------------------------------------------------------------------------------------------------------------
sheet = wb['Ежемесячно_Кампании']

for i in ['A2','B2','C2','D2','E2','F2','G2','H2','I2']:
    cell = sheet[i]

    cell.alignment = Alignment(wrapText=True,horizontal='center',vertical='center')


thin_border = Border(left=Side(style='thin',color='006400'),
                     right=Side(style='thin',color='006400'),
                     top=Side(style='thin',color='006400'),
                     bottom=Side(style='thin',color='006400'))


for row in range(2, sheet.max_row + 1):
    for col in range(1, sheet.max_column + 1):
        sheet.cell(row=row, column=col).border = thin_border

#--------------------------------------------------------------------------------------------------------------------------------------------------------------

sheet = wb['Воронка отсечений']

for i in ['A2','B2','C2','D2','E2','F2','G2','H2','I2','J2','K2','L2','M2']:
    cell = sheet[i]

    cell.alignment = Alignment(wrapText=True,horizontal='center',vertical='center')


thin_border = Border(left=Side(style='thin',color='006400'),
                     right=Side(style='thin',color='006400'),
                     top=Side(style='thin',color='006400'),
                     bottom=Side(style='thin',color='006400'))


for row in range(2, sheet.max_row + 1):
    for col in range(1, sheet.max_column + 1):
        sheet.cell(row=row, column=col).border = thin_border

#--------------------------------------------------------------------------------------------------------------------------------------------------------------

sheet = wb['Стабильность передачи базы']

for i in ['A2','B2','C2','D2']:
    cell = sheet[i]

    cell.alignment = Alignment(wrapText=True,horizontal='center',vertical='center')


thin_border = Border(left=Side(style='thin',color='006400'),
                     right=Side(style='thin',color='006400'),
                     top=Side(style='thin',color='006400'),
                     bottom=Side(style='thin',color='006400'))


for row in range(2, sheet.max_row + 1):
    for col in range(1, sheet.max_column + 1):
        sheet.cell(row=row, column=col).border = thin_border

#--------------------------------------------------------------------------------------------------------------------------------------------------------------
# Сохраните изменения в файл
wb.save('excl.xlsx')